In [ ]:
import numpy as np
import pandas as pd
import ta
import ta
import yfinance as yf
from time import time

In [2]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_FAST_PERIOD = 12
MACD_SLOW_PERIOD = 26
MACD_SIGNAL_PERIOD = 9

In [3]:
def calculate_indicators_macd(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['Close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)
    
    return df

In [4]:
def trading_strategy_test_macd(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Previous_MACD'] < df['Previous_Signal_Line']) & (df['MACD'] >= df['Signal_Line']) & (df['RSI'] > RSI_OVERSOLD)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['Close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['Close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [5]:
start_date='2014-01-01'
end_date='2016-01-01'

In [6]:
non_ESG = ["GME", "FOSL", "GT", "HOG", "PDCO"]

In [7]:
high_ESG =  ['CCU','CNHI.VI','TIETO.HE','BAYN.BE','SGSNZ.XC']

In [8]:
medium_esg = ['VOC','VRSK','19MA.F','QNBK.QA','1605.TW']

In [9]:
all_comp = high_ESG + medium_esg + non_ESG

In [10]:
def company_winrate(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)
    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_macd(df)
    winrate = trading_strategy_test_macd(df)
        
    return winrate

In [ ]:
winrate_macd = {}
for company in all_comp:
    time.sleep(10)
    try:
        company_winr = company_winrate(company, start_date=start_date, end_date=end_date)
        winrate_macd[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

Error occurred for CCU: Too Many Requests. Rate limited. Try after a while.
Error occurred for CNHI.VI: Too Many Requests. Rate limited. Try after a while.


KeyboardInterrupt: 

In [ ]:
winrate_macd

{'cop': {5: 28.57142857142857,
  10: 28.57142857142857,
  15: 57.14285714285714,
  20: 50.0,
  25: 66.66666666666666,
  30: 66.66666666666666},
 'msft': {5: 50.0, 10: 75.0, 15: 87.5, 20: 100.0, 25: 100.0, 30: 87.5},
 'eix': {5: 75.0,
  10: 62.5,
  15: 75.0,
  20: 85.71428571428571,
  25: 71.42857142857143,
  30: 71.42857142857143},
 'abnb': {5: 50.0,
  10: 66.66666666666666,
  15: 66.66666666666666,
  20: 50.0,
  25: 66.66666666666666,
  30: 50.0},
 'apd': {5: 12.5,
  10: 62.5,
  15: 42.857142857142854,
  20: 28.57142857142857,
  25: 28.57142857142857,
  30: 42.857142857142854},
 'amzn': {5: 57.14285714285714,
  10: 71.42857142857143,
  15: 85.71428571428571,
  20: 57.14285714285714,
  25: 42.857142857142854,
  30: 57.14285714285714},
 'nem': {5: 60.0,
  10: 50.0,
  15: 60.0,
  20: 55.55555555555556,
  25: 55.55555555555556,
  30: 55.55555555555556},
 'pep': {5: 40.0,
  10: 50.0,
  15: 60.0,
  20: 66.66666666666666,
  25: 77.77777777777779,
  30: 88.88888888888889},
 'exc': {5: 63.6363

In [ ]:
winrate_macd = pd.DataFrame(winrate_macd)

In [ ]:
winrate_macd.to_excel('winrate_result/macd.xlsx')

In [ ]:
def calculate_indicators_bb(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['Close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['Close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    return df

In [ ]:
def trading_strategy_test_bb(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Close'] <= df['Bollinger_low']) & (df['RSI'] < RSI_OVERSOLD)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['Close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['Close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [ ]:
def company_winrate_bb(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)

    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_bb(df)
    winrate = trading_strategy_test_bb(df)
        
    return winrate

In [ ]:
winrate_bb = {}
for company in all_comp:
    time.sleep(12)
    try:
        company_winr = company_winrate_bb(company, start_date=start_date, end_date=end_date)
        winrate_bb[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_21288\402886830.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21288\402886830.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [ ]:
winrate_bb

{'cop': {5: 0, 10: 0, 15: 0, 20: 0, 25: 0, 30: 0},
 'msft': {5: 0, 10: 0, 15: 0, 20: 0, 25: 0, 30: 0},
 'eix': {5: 50.0, 10: 100.0, 15: 75.0, 20: 75.0, 25: 100.0, 30: 100.0},
 'abnb': {5: 0.0, 10: 0.0, 15: 25.0, 20: 50.0, 25: 50.0, 30: 50.0},
 'apd': {5: 66.66666666666666,
  10: 33.33333333333333,
  15: 100.0,
  20: 100.0,
  25: 100.0,
  30: 100.0},
 'amzn': {5: 100.0, 10: 100.0, 15: 100.0, 20: 100.0, 25: 100.0, 30: 100.0},
 'nem': {5: 83.33333333333334,
  10: 66.66666666666666,
  15: 50.0,
  20: 33.33333333333333,
  25: 33.33333333333333,
  30: 33.33333333333333},
 'pep': {5: 0, 10: 0, 15: 0, 20: 0, 25: 0, 30: 0},
 'exc': {5: 100.0, 10: 100.0, 15: 100.0, 20: 100.0, 25: 100.0, 30: 100.0},
 'tt': {5: 100.0, 10: 100.0, 15: 100.0, 20: 100.0, 25: 100.0, 30: 100.0},
 'pnw': {5: 83.33333333333334,
  10: 66.66666666666666,
  15: 50.0,
  20: 50.0,
  25: 50.0,
  30: 66.66666666666666},
 'sre': {5: 100.0, 10: 100.0, 15: 100.0, 20: 100.0, 25: 100.0, 30: 100.0},
 'mpc': {5: 66.66666666666666,
  10

In [ ]:
winrate_bb = pd.DataFrame(winrate_bb)

In [ ]:
winrate_bb.to_excel('winrate_result/bb.xlsx')

In [ ]:
OBV_PERIOD = 5

In [ ]:
def calculate_indicators_obv(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [ ]:
def trading_strategy_test_obv(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['Close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['Close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [ ]:
def company_winrate_obv(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)

    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_obv(df)
    winrate = trading_strategy_test_obv(df)
        
    return winrate

In [ ]:
winrate_obv = {}
for company in all_comp:
    time.sleep(12)
    try:
        company_winr = company_winrate_obv(company, start_date=start_date, end_date=end_date)
        winrate_obv[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_21288\1922382557.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21288\1922382557.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [ ]:
winrate_obv = pd.DataFrame(winrate_obv)
winrate_obv.to_excel('winrate_result/obv.xlsx')